In [5]:
import pandas as pd
import re
import numpy as np
from geopy.geocoders import Nominatim
import folium
nom = Nominatim(user_agent="Windows")

In [6]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [2]:
df1 = pd.read_csv('idealista_alq.csv', delimiter=',')

In [3]:
df1.head()

,URL,adress,describe,detail_1,detail_2,ft,img_urls,latitude,longitude,price,stat_text,tel,title,title_minor,txt_diposit,vender_name,vender_type
0,https://www.idealista.com/inmueble/1684608/?xt...,"Callejón Pere Codina i Mont, 57 , Distrito Fe...","""-Piso ático impecable, alegre y muy luminoso...","80 m² construidos,2 habitaciones,1 baño,Terraz...","Planta 4ª exterior , Con ascensor , Piscina ,...","80 m²,2 hab.,4ª planta exterior con ascensor",NaN,41.698042,2.828084,750,Anuncio actualizado el 30 de diciembre,NaN,Alquiler de Piso en callejón Pere Codina i Mon...,"Fenals, Lloret de Mar",NaN,jesús,Particular
1,https://www.idealista.com/inmueble/83783412/?x...,"Avenida de Fenals, 10 , Urb. santa clotilde ,...",NaN,"35 m² construidos, 30 m² útiles,Sin habitación...","Bajo exterior , Con ascensor , Aire acondicio...","35 m²,Bajo exterior con ascensor",NaN,41.695645,2.822507,450,Anuncio actualizado el 9 de enero,695975520.0,"Alquiler de Estudio en avenida de Fenals, 10","Santa Clotilde, Lloret de Mar",NaN,sergio benjamin,Particular
2,https://www.idealista.com/inmueble/35464081/?x...,"Avenida Joan Llaverias, 49 , Distrito Casc An...","""Apartamento, exterior, tranquilo, bien comun...","45 m² construidos, 40 m² útiles,1 habitación,1...","Planta 1ª exterior , Con ascensor , Aire acon...","45 m²,1 hab.,1ª planta exterior con ascensor",NaN,41.701183,2.854635,550,Anuncio actualizado el 20 de febrero,616172915.0,"Alquiler de Piso en avenida Joan Llaverias, 49","Casc Antic, Lloret de Mar",Fianza de 2 meses,Sole,Particular
3,https://www.idealista.com/inmueble/84095573/?x...,"Calle Montecarlo, 44 , Urb. font de sant llor...",NaN,"Casa o chalet independiente,2 plantas,140 m² ...","Aire acondicionado , Piscina , Jardín","140 m²,3 hab.",NaN,41.712568,2.881266,8000,Anuncio actualizado el 31 de enero,NaN,Alquiler de Casa o chalet independiente en cal...,"Serra Brava, Lloret de Mar",NaN,paco,Particular
4,https://www.idealista.com/inmueble/83290881/?x...,"Calle Costa d'en Bandina , Distrito Canyelles...","""Preciosa casa situada a 800 m del mar. Excel...","Casa o chalet,2 plantas,200 m² construidos, 1...","Aire acondicionado , Jardín","200 m²,6 hab.,Garaje incluido",NaN,41.709277,2.877646,1650,Anuncio actualizado el 24 de enero,931342518.0,Alquiler de Casa o chalet en calle Costa d'en ...,"Canyelles - Montgoda, Lloret de Mar",NaN,Oi Realtor Lloret,Profesional


In [10]:
data = df1.drop((['URL', 'img_urls', ]), axis=1)

In [11]:
data.price.median(), data.price.mean()

(630.0, 920.3703703703703)

In [20]:
missing_data(data)

,adress,describe,detail_1,detail_2,ft,latitude,longitude,price,stat_text,tel,title,title_minor,txt_diposit,vender_name,vender_type,m2,hab
Total,0,2,0,4,0,0,0,0,0,2,0,0,29,0,0,0,0
Percent,0,3.7037,0,7.40741,0,0,0,0,0,3.7037,0,0,53.7037,0,0,0,0
Types,object,object,object,object,object,float64,float64,int64,object,float64,object,object,object,object,object,object,object


In [ ]:
s = nom.geocode(language='ES', query='lloret de mar')
lloret_coordinates = [s.latitude, s.longitude]
dfmap = df1[['latitude', 'longitude', 'price', 'adress']]
dfmap.head()

In [14]:
map_casa = folium.Map(location=lloret_coordinates, tiles='OpenStreetMap', zoom_start=14)

In [ ]:
for i in range(dfmap.latitude.count()):
    folium.CircleMarker([dfmap.latitude[i],dfmap.longitude[i]], 
                        radius=5, 
                        color='blue', 
                        #popup=dfmap['price'], 
                        fill=True).add_to(map_casa)

In [15]:
for each in dfmap.iterrows():
    folium.CircleMarker([each[1]['latitude'],each[1]['longitude']],
                        radius=5, color='blue',
                        popup=str(each[1]['price'] + adress),
                        fill=True).add_to(map_casa)

In [ ]:
map_casa

In [26]:
data.head(10)

,adress,describe,detail_1,detail_2,ft,latitude,longitude,price,stat_text,tel,title,title_minor,txt_diposit,vender_name,vender_type,m2,hab
0,"Callejón Pere Codina i Mont, 57 , Distrito Fe...","""-Piso ático impecable, alegre y muy luminoso...","80 m² construidos,2 habitaciones,1 baño,Terraz...","Planta 4ª exterior , Con ascensor , Piscina ,...","80 m²,2 hab.,4ª planta exterior con ascensor",41.698042,2.828084,750,Anuncio actualizado el 30 de diciembre,NaN,Alquiler de Piso en callejón Pere Codina i Mon...,"Fenals, Lloret de Mar",NaN,jesús,Particular,80,[2]
1,"Avenida de Fenals, 10 , Urb. santa clotilde ,...",NaN,"35 m² construidos, 30 m² útiles,Sin habitación...","Bajo exterior , Con ascensor , Aire acondicio...","35 m²,Bajo exterior con ascensor",41.695645,2.822507,450,Anuncio actualizado el 9 de enero,695975520.0,"Alquiler de Estudio en avenida de Fenals, 10","Santa Clotilde, Lloret de Mar",NaN,sergio benjamin,Particular,35,[]
2,"Avenida Joan Llaverias, 49 , Distrito Casc An...","""Apartamento, exterior, tranquilo, bien comun...","45 m² construidos, 40 m² útiles,1 habitación,1...","Planta 1ª exterior , Con ascensor , Aire acon...","45 m²,1 hab.,1ª planta exterior con ascensor",41.701183,2.854635,550,Anuncio actualizado el 20 de febrero,616172915.0,"Alquiler de Piso en avenida Joan Llaverias, 49","Casc Antic, Lloret de Mar",Fianza de 2 meses,Sole,Particular,45,[1]
3,"Calle Montecarlo, 44 , Urb. font de sant llor...",NaN,"Casa o chalet independiente,2 plantas,140 m² ...","Aire acondicionado , Piscina , Jardín","140 m²,3 hab.",41.712568,2.881266,8000,Anuncio actualizado el 31 de enero,NaN,Alquiler de Casa o chalet independiente en cal...,"Serra Brava, Lloret de Mar",NaN,paco,Particular,140,[3]
4,"Calle Costa d'en Bandina , Distrito Canyelles...","""Preciosa casa situada a 800 m del mar. Excel...","Casa o chalet,2 plantas,200 m² construidos, 1...","Aire acondicionado , Jardín","200 m²,6 hab.,Garaje incluido",41.709277,2.877646,1650,Anuncio actualizado el 24 de enero,931342518.0,Alquiler de Casa o chalet en calle Costa d'en ...,"Canyelles - Montgoda, Lloret de Mar",NaN,Oi Realtor Lloret,Profesional,200,[6]
5,", Distrito Casc Antic , Lloret de Mar , La Se...","""Alquiler temporal de bonito piso en Lloret d...","106 m² construidos, 90 m² útiles,3 habitacione...","Planta 4ª , Con ascensor","106 m²,3 hab.,4ª planta con ascensor,Garaje in...",41.702508,2.855645,700,Anuncio actualizado el 30 de enero,935438356.0,Alquiler de Piso en Casc Antic,Lloret de Mar,NaN,Luxury V. B&G,Profesional,106,[3]
6,"Calle Joaquim Codina i Vinyes , Distrito Sant...","""GIRONA - LLORET DE MAR,JOAQUIM CODINA I VINY...","Casa o chalet,140 m² construidos,4 habitacion...","Piscina , Jardín","140 m²,4 hab.,Garaje incluido",41.695369,2.826362,1200,Anuncio actualizado el 11 de febrero,972939625.0,Alquiler de Casa o chalet en calle Joaquim Cod...,"Santa Clotilde, Lloret de Mar",NaN,Débora,Profesional,140,[4]
7,"Calle del Migdia , Distrito Casc Antic , Llor...","""Altura piso 1º, piso superficie total 39 m²,...","39 m² construidos, 35 m² útiles,1 habitación,1...","Planta 1ª exterior , Sin ascensor","39 m²,1 hab.,1ª planta exterior sin ascensor",41.703557,2.849015,390,Anuncio actualizado el 25 de febrero,972935628.0,Alquiler de Piso en calle del Migdia,"Casc Antic, Lloret de Mar",NaN,CAT HABITAT,Profesional,39,[1]
8,", Distrito Casc Antic , Lloret de Mar , La Se...","""Fantástico ático en alquiler en el semi-cent...","160 m² construidos, 70 m² útiles,2 habitacione...","Planta 5ª , Con ascensor","160 m²,2 hab.,5ª planta con ascensor",41.703131,2.853711,700,Anuncio actualizado el 25 de febrero,972080573.0,Alquiler de Piso en Casc Antic,Lloret de Mar,NaN,ISMAEL PÉREZ,Profesional,160,[2]
9,"Rambla de Roma Barnes, 10 , Distrito Casc Ant...","""ACOGEDOR APARTAMENTO/ESTUDIO SEGUNDA LÍNEA D...","55 m² construidos,1 habitación,1 baño,Segunda ...","Planta 1ª exterior , Sin ascensor , Aire acon...","55 m²,1 hab.,1ª planta exterior sin ascensor",41.698291,2.845731,425,Anuncio actu

In [42]:
data['hab'] = df1['ft'].apply(lambda x: re.findall(r'(\d+) hab.', x))

In [25]:
data['m2'] = df1['ft'].apply(lambda x: re.findall(r'(\d\d+) m²', x)[0])

In [46]:
data['hab'] = data['hab'].apply(lambda row: row.replace(']', ''))

AttributeError: 'list' object has no attribute 'replace'